In [1]:
import tensorflow as tf
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 添加層def add_layer()

In [2]:
def add_layer(inputs, in_size, out_size,n_layer, activation_function=None):
    layer_name='layer%s'%(n_layer)
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights= tf.Variable(tf.random_normal([in_size, out_size]),name='W')
            tf.summary.histogram(layer_name + '/weights', Weights) # tensorflow >= 0.12
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1,name='b')
            tf.summary.histogram(layer_name + '/biases', biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights),biases)
            tf.summary.histogram(layer_name + '/Wx_plus_b', Wx_plus_b)

        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name + '/outputs', outputs)
        return outputs

# 導入數據

In [3]:
x_data = np.linspace(-1,1,300, dtype=np.float32)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape).astype(np.float32)
y_data = np.square(x_data) - 0.5 + noise

# 神經網絡的輸入

In [4]:
with tf.name_scope('inputs'):
    # define placeholder for inputs to network
    xs = tf.placeholder(tf.float32, [None, 1])
    ys = tf.placeholder(tf.float32, [None, 1])

# 搭建網絡

In [5]:
l1 = add_layer(xs, 1, 10,n_layer=1, activation_function=tf.nn.relu)
prediction = add_layer(l1, 10, 1, n_layer=2, activation_function=None)
# the error between prediciton and real data
with tf.name_scope('loss'):    
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),reduction_indices=[1]))
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
init = tf.global_variables_initializer()  # 替换成这样就好
sess = tf.Session()
merge = tf.summary.merge_all()
writer = tf.summary.FileWriter("logs/", sess.graph)
sess.run(init)

# 訓練

In [6]:
for i in range(1000):
    # training
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i % 50 == 0:
        # to see the step improvement
        result = sess.run(merge, feed_dict={xs: x_data, ys: y_data})
        writer.add_summary(result,i)
        print(sess.run(loss, feed_dict={xs: x_data, ys: y_data}))

0.31663626
0.021217814
0.012471888
0.008579059
0.006359215
0.005189775
0.0046022707
0.0043624113
0.0041791177
0.0040099677
0.0038600059
0.003745575
0.003652386
0.0035801074
0.0035153127
0.0034611747
0.0034203113
0.003380406
0.0033515552
0.003321291
